In [ ]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"


In [ ]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from nb_quantfreedom.nb_enums import CandleBodyType

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
from numba import int64, types, typed
from numba.experimental import jitclass

spec_foo = {"data": int64[:]}


@jitclass(spec_foo)
class Foo(object):
    def __init__(self, init_data):
        self.data = init_data


# Want to have a list of Foo instances in the Bar class
# >>> Use ListType instead of List here.
# spec_bar = {"foo_list": types.ListType(Foo.class_type.instance_type)}


# @jitclass(spec_bar)
class Bar(object):
    def __init__(self, foo_list):
        # This assigns a reference to a list of Foo instances
        self.foo_list = foo_list


x = np.arange(5).astype(np.int64)
# Build a Foo instance
foo = Foo(x)

# Create a Bar instance that holds a list of Foo instance.
# >>> Pass a typed.List here.
bar = Bar(typed.List([foo]))
foo

In [ ]:
bar.foo_list[0].data

In [ ]:
foo.data

In [107]:
import numba
import numpy as np
from numba import int64, types, typed, optional
from numba.experimental import jitclass

spec = {"data": int64}


class EmptyClass:
    def __init__(self) -> None:
        pass

    def tester(self):
        print("hello")

    def multi_data(self):
        pass

    def print_data(self):
        pass


@jitclass(spec)
class Foo(EmptyClass):
    def __init__(self, init_data):
        self.data = init_data

    def print_data(self):
        print(self.data)


@jitclass(spec)
class Boo(EmptyClass):
    def __init__(self, init_data):
        self.data = init_data

    def multi_data(self):
        print(self.data * 100)


spec_end = {"logger": numba.types.UnionType((Foo.class_type.instance_type, Boo.class_type.instance_type))}


@jitclass(spec_end)
class Bar:
    def __init__(self, data):
        if data == 0:
            self.logger = Foo(1)
        elif data == 1:
            self.logger = Boo(2)

In [108]:
Bar(0)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: native lowering)
[1m[1mCannot cast instance.jitclass.Foo#1d61f31b6d0<data:int64> to Union[instance.jitclass.Boo#1d61f318f70<data:int64>,instance.jitclass.Foo#1d61f31b6d0<data:int64>]: %".63" = load {i8*, {i64}*}, {i8*, {i64}*}* %".31"[0m
[0m[1mDuring: lowering "(self).logger = $14call_function.2" at C:\Users\User\AppData\Local\Temp\ipykernel_19292\1499562187.py (56)[0m
[0m[1mDuring: resolving callee type: jitclass.Bar#1d61f1cddb0<logger:Union[instance.jitclass.Boo#1d61f318f70<data:int64>,instance.jitclass.Foo#1d61f31b6d0<data:int64>]>[0m
[0m[1mDuring: typing of call at <string> (3)
[0m
[0m[1mDuring: resolving callee type: jitclass.Bar#1d61f1cddb0<logger:Union[instance.jitclass.Boo#1d61f318f70<data:int64>,instance.jitclass.Foo#1d61f31b6d0<data:int64>]>[0m
[0m[1mDuring: typing of call at <string> (3)
[0m
[1m
File "<string>", line 3:[0m
[1m<s

In [115]:
import numpy as np
import numba as nb

@nb.njit()
def create_date(time_since_epoch):
    epoch = np.datetime64('1970-01-01', 'ms')
    td_unit = np.timedelta64(1, 'ms')
    return epoch + time_since_epoch * td_unit

print(create_date(1697884200000))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(datetime64[]) found for signature:
 
 >>> <unknown function>(Literal[str](1970-01-01), Literal[str](ms))
 
There are 2 candidate implementations:
[1m - Of which 2 did not match due to:
 Overload in function 'make_callable_template.<locals>.generic': File: numba\core\typing\templates.py: Line 174.
   With argument(s): '(unicode_type, unicode_type)':[0m
[1m  Rejected as the implementation raised a specific error:
    TypingError: [1mtoo many positional arguments[0m[0m
  raised from e:\Coding\venvs\qfree\lib\site-packages\numba\core\typing\templates.py:412
[0m
[0m[1mDuring: resolving callee type: class(datetime64[])[0m
[0m[1mDuring: typing of call at C:\Users\User\AppData\Local\Temp\ipykernel_19292\890831128.py (6)
[0m
[1m
File "C:\Users\User\AppData\Local\Temp\ipykernel_19292\890831128.py", line 6:[0m
[1mdef create_date(time_since_epoch):
[1m    epoc

In [ ]:
numba.types